In [2]:
import pandas as pd
import json

In [3]:
data_file = "/Users/saral/Documents/cricket/All data/t20s_json/1503460.json"

# Load JSON
with open(data_file, "r") as f:
    data = json.load(f)

rows = []

# Iterate over innings
for idx, inng in enumerate(data["innings"], start=1):
    team = inng["team"]

    for over_data in inng["overs"]:
        over = over_data["over"]

        for ball_number, delivery in enumerate(over_data["deliveries"], start=1):

            runs = delivery.get("runs", {})
            extras = delivery.get("extras", {})

            # Determine extra type
            if "wides" in extras:
                extra_type = "wide"
            elif "noballs" in extras:
                extra_type = "no_ball"
            elif "legbyes" in extras:
                extra_type = "legbye"
            elif "byes" in extras:
                extra_type = "bye"
            else:
                extra_type = None

            # Wicket details
            wicket_flag = "wickets" in delivery
            if wicket_flag:
                wk = delivery["wickets"][0]
                player_out = wk.get("player_out")
                wicket_kind = wk.get("kind")
                fielders = [f["name"] for f in wk.get("fielders", [])] if "fielders" in wk else []
            else:
                player_out = None
                wicket_kind = None
                fielders = []

            rows.append({
                "innings_number": idx,
                "batting_team": team,
                "over": over,
                "ball_in_over": ball_number,
                "batter": delivery.get("batter"),
                "non_striker": delivery.get("non_striker"),
                "bowler": delivery.get("bowler"),
                "runs_batter": runs.get("batter", 0),
                "runs_extras": runs.get("extras", 0),
                "runs_total": runs.get("total", 0),
                "extra_type": extra_type,
                "is_wicket": wicket_flag,
                "wicket_player_out": player_out,
                "wicket_type": wicket_kind,
                "wicket_fielders": fielders
            })

df = pd.DataFrame(rows)
df.head()


,innings_number,batting_team,over,ball_in_over,batter,non_striker,bowler,runs_batter,runs_extras,runs_total,extra_type,is_wicket,wicket_player_out,wicket_type,wicket_fielders
0,1,Nepal,0,1,K Bhurtel,Aasif Sheikh,Junaid Siddique,0,1,1,wide,False,None,None,[]
1,1,Nepal,0,2,K Bhurtel,Aasif Sheikh,Junaid Siddique,0,0,0,None,False,None,None,[]
2,1,Nepal,0,3,K Bhurtel,Aasif Sheikh,Junaid Siddique,1,0,1,None,False,None,None,[]
3,1,Nepal,0,4,Aasif Sheikh,K Bhurtel,Junaid Siddique,1,0,1,None,False,None,None,[]
4,1,Nepal,0,5,K Bhurtel,Aasif Sheikh,Junaid Siddique,0,0,0,None,False,None,None,[]


In [4]:
# create df where innings_number is 1
df_innings_1 = df[df["innings_number"] == 1]
# create df where innings_number is 2
df_innings_2 = df[df["innings_number"] == 2]

In [5]:
df_innings_1

,innings_number,batting_team,over,ball_in_over,batter,non_striker,bowler,runs_batter,runs_extras,runs_total,extra_type,is_wicket,wicket_player_out,wicket_type,wicket_fielders
0,1,Nepal,0,1,K Bhurtel,Aasif Sheikh,Junaid Siddique,0,1,1,wide,False,None,None,[]
1,1,Nepal,0,2,K Bhurtel,Aasif Sheikh,Junaid Siddique,0,0,0,None,False,None,None,[]
2,1,Nepal,0,3,K Bhurtel,Aasif Sheikh,Junaid Siddique,1,0,1,None,False,None,None,[]
3,1,Nepal,0,4,Aasif Sheikh,K Bhurtel,Junaid Siddique,1,0,1,None,False,None,None,[]
4,1,Nepal,0,5,K Bhurtel,Aasif Sheikh,Junaid Siddique,0,0,0,None,False,None,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,1,Nepal,19,2,S Jora,Sompal Kami,Junaid Siddique,1,0,1,None,False,None,None,[]
118,1,Nepal,19,3,Sompal Kami,S Jora,Junaid Siddique,2,0,2,None,False,None,None,[]
119,1,Nepal,19,4,Sompal Kami,S Jora,Junaid Siddique,1,0,1,None,False,None,None,[]
120,1,Nepal,19,5,S Jora,Sompal Kami,Junaid Siddique,0,1,1,bye,False,None,None,[]


In [6]:
df_innings_2['extra_type'].value_counts()


extra_type
wide      3
legbye    1
Name: count, dtype: int64

In [7]:
import pandas as pd
import numpy as np

def make_scorecard(df_innings):
    # keep ball_id for batting order
    df = df_innings.reset_index().rename(columns={'index': 'ball_id'})

    # balls faced: exclude wides
    balls_df = df[df['extra_type'] != 'wide'].copy()

    # per-batter aggregation
    scorecard = (
        balls_df.groupby("batter").agg(
            Runs=("runs_batter", "sum"),
            Balls=("runs_batter", "size"),
            Fours=("runs_batter", lambda x: (x == 4).sum()),
            Sixes=("runs_batter", lambda x: (x == 6).sum()),
            Dots=("runs_batter",  lambda x: (x == 0).sum()),
        )
        .reset_index()
        .rename(columns={"batter": "Batsman"})
    )

    # batting order by first appearance
    order = (
        df.groupby("batter")["ball_id"]
        .min()
        .reset_index()
        .rename(columns={"batter": "Batsman", "ball_id": "first_ball"})
    )

    scorecard = (
        scorecard
        .merge(order, on="Batsman", how="left")
        .sort_values("first_ball")
        .drop(columns="first_ball")
    )

    # strike rate & dot percentage per batter
    scorecard["SR"] = (scorecard["Runs"] * 100 / scorecard["Balls"]).round(2)
    scorecard["Dot%"] = (scorecard["Dots"] * 100 / scorecard["Balls"]).round(0)

    # --- totals (batters only) ---
    total_runs_bat = scorecard["Runs"].sum()
    total_balls    = scorecard["Balls"].sum()
    total_fours    = scorecard["Fours"].sum()
    total_sixes    = scorecard["Sixes"].sum()
    total_dots     = scorecard["Dots"].sum()

    extras_total = df_innings["runs_extras"].sum()
    team_total   = total_runs_bat + extras_total

    team_sr   = round(team_total * 100 / total_balls, 2) if total_balls > 0 else 0
    team_dotp = round(total_dots * 100 / total_balls, 0) if total_balls > 0 else 0

    # TOTAL row (batters only)
    total_row = pd.DataFrame([{
        "Batsman": "TOTAL",
        "Runs": total_runs_bat,
        "Balls": total_balls,
        "Fours": total_fours,
        "Sixes": total_sixes,
        "Dots": total_dots,
        "Dot%": team_dotp,
        "SR": ""
    }])

    # EXTRAS row
    extras_row = pd.DataFrame([{
        "Batsman": "EXTRAS",
        "Runs": extras_total,
        "Balls": "",
        "Fours": "",
        "Sixes": "",
        "Dots": "",
        "Dot%": "",
        "SR": "",
    }])

    # GRAND TOTAL row (batters + extras)
    grand_row = pd.DataFrame([{
        "Batsman": "GRAND TOTAL",
        "Runs": team_total,
        "Balls": total_balls,
        "Fours": total_fours,
        "Sixes": total_sixes,
        "Dots": total_dots,
        "Dot%": team_dotp,
        "SR": team_sr,
    }])

    # combine
    scorecard = pd.concat([scorecard, total_row, extras_row, grand_row], ignore_index=True)

    return scorecard


In [8]:
scorecard_1 = make_scorecard(df_innings_1)
scorecard_2 = make_scorecard(df_innings_2)



In [9]:
scorecard_1


,Batsman,Runs,Balls,Fours,Sixes,Dots,SR,Dot%
0,K Bhurtel,28,26,2,2,14,107.69,54.0
1,Aasif Sheikh,51,47,4,1,17,108.51,36.0
2,RK Paudel,20,19,1,0,6,105.26,32.0
3,Kushal Malla,10,11,0,0,3,90.91,27.0
4,DS Airee,1,2,0,0,1,50.0,50.0
5,Gulsan Jha,14,5,2,1,2,280.0,40.0
6,Sompal Kami,6,6,0,0,1,100.0,17.0
7,S Jora,4,4,0,0,1,100.0,25.0
8,TOTAL,134,120,9,4,45,,38.0
9,EXTRAS,6,,,,,,


In [10]:
scorecard_2

,Batsman,Runs,Balls,Fours,Sixes,Dots,SR,Dot%
0,J Figy John,19,29,1,1,18,65.52,62.0
1,Waseem Muhammad,1,3,0,0,2,33.33,67.0
2,A Sharafu,58,40,3,5,17,145.0,42.0
3,R Chopra,7,11,0,0,5,63.64,45.0
4,H Kaushik,2,5,0,0,3,40.0,60.0
5,D Parashar,32,18,1,3,5,177.78,28.0
6,Basil Hameed,0,1,0,0,1,0.0,100.0
7,Zahid Ali,11,11,0,0,0,100.0,0.0
8,Haider Ali,0,1,0,0,1,0.0,100.0
9,Junaid Siddique,1,1,0,0,0,100.0,0.0


In [11]:
## innings without wides and no-balls
df_1 = df_innings_1[(df_innings_1['extra_type'] != 'wide') & (df_innings_1['extra_type'] != 'no-ball')].copy()
df_2 = df_innings_2[(df_innings_2['extra_type'] != 'wide') & (df_innings_2['extra_type'] != 'no-ball')].copy()


In [12]:
# runs in the first 6 overs, wickets lost print with /  here wides are important and noballs
print(f"{df_innings_1[df_innings_1.over < 6].runs_total.sum()} / {df_innings_1[(df_innings_1.over < 6) & df_innings_1.is_wicket].shape[0]}")
print(f"{df_innings_2[df_innings_2.over < 6].runs_total.sum()} / {df_innings_2[(df_innings_2.over < 6) & df_innings_2.is_wicket].shape[0]}")


45 / 0
40 / 1


In [13]:
#runs from 7-15 overs and wicket lost in the same 
print(f"{df_innings_1[(df_innings_1.over >= 6) & (df_innings_1.over <=15)]['runs_total'].sum()} / {df_innings_1[(df_innings_1.over >= 6) & (df_innings_1.over <=15) & (df_innings_1.is_wicket) ].shape[0]}")
print(f"{df_innings_2[(df_innings_2.over >= 6) & (df_innings_2.over <=15)]['runs_total'].sum()} / {df_innings_2[(df_innings_2.over >= 6) & (df_innings_2.over <=15) & (df_innings_2.is_wicket) ].shape[0]}")  

58 / 3
59 / 5


In [14]:
# runs in the last 5 overs wickets lost
print(f"{df_innings_1[df_innings_1.over > 15].runs_total.sum()} / {df_innings_1[(df_innings_1.over > 15) & df_innings_1.is_wicket].shape[0]}")
print(f"{df_innings_2[df_innings_2.over > 15].runs_total.sum()} / {df_innings_2[(df_innings_2.over > 15) & df_innings_2.is_wicket].shape[0]}")

37 / 3
40 / 3


In [15]:
# Dot = no run AND not a wide or no-ball
df_innings_1["is_dot"] = (
        ((df_innings_1["runs_batter"] == 0) | 
         (df_innings_1["extra_type"].isin(["bye", "legbye"]))) &
        (~df_innings_1["extra_type"].isin(["wide", "no_ball"]))
    )



/var/folders/qx/d6vwfn4179979vr76dn4k3440000gn/T/ipykernel_89533/3994640395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_innings_1["is_dot"] = (


In [16]:
df_innings_2["is_dot"] = (
        ((df_innings_2["runs_batter"] == 0) | 
         (df_innings_2["extra_type"].isin(["bye", "legbye"]))) &
        (~df_innings_2["extra_type"].isin(["wide", "no_ball"]))
    )

/var/folders/qx/d6vwfn4179979vr76dn4k3440000gn/T/ipykernel_89533/4134929733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_innings_2["is_dot"] = (


In [17]:
# Dot balls count for all overs
df_innings_1["is_dot"].sum()


np.int64(45)

In [18]:
# check power play dot balls for innings 1
print(df_innings_1[df_innings_1.over < 6].is_dot.sum())
# for middle overs (starting the 7 over to the 15th over)
print(df_innings_1[(df_innings_1.over >=6) & (df_innings_1.over <=15)].is_dot.sum())
# for death overs (Starting from 16th over to the 20th over)
print(df_innings_1[df_innings_1.over >15].is_dot.sum())

17
21
7


In [19]:
# check power play dot balls for innings 2
print(df_innings_2[df_innings_2.over < 6].is_dot.sum())
# for middle overs
print(df_innings_2[(df_innings_2.over >=6) & (df_innings_2.over <=15)].is_dot.sum())
# for death overs
print(df_innings_2[df_innings_2.over >15].is_dot.sum())

24
24
4


In [20]:
# show total 4 runs runs_batter that were fours in power play overs for innings 2
print(df_innings_2[(df_innings_2.over < 6) ].runs_batter.sum())

# print(df_innings_2[(df_innings_2.over < 6) & (df_innings_2.runs_batter == 4)].runs_batter.sum())
# for middle overs
# print(df_innings_2[(df_innings_2.over >=6) & (df_innings_2.over <=15)].is_dot.sum())
# # for death overs
# print(df_innings_2[df_innings_2.over >15].is_dot.sum())

33


In [21]:
# Nepal Innings 2 powerplay runs_batter value counts
pp_2 = pd.DataFrame(df_2[df_2.over < 6].runs_batter.value_counts())
# South Africa Innings in middle overs
md_2 = pd.DataFrame(df_2[(df_2.over >= 6) & (df_2.over <= 15)].runs_batter.value_counts())
# # South Africa Innings in death overs

dt_2 = pd.DataFrame(df_2[df_2.over > 15].runs_batter.value_counts())

In [22]:
# merge pp_1, md_1, dt_1 dataframes
runs_summary_2 = pp_2.merge(md_2, left_index=True, right_index=True, how='outer', suffixes=('_PP', '_MD'))
runs_summary_2 = runs_summary_2.merge(dt_2, left_index=True, right_index=True, how='outer')
runs_summary_2 = runs_summary_2.rename(columns={0: 'DT'})
runs_summary_2 = runs_summary_2.fillna(0).astype(int)
runs_summary_2

,count_PP,count_MD,count
runs_batter,,,
0,24,24,4
1,7,29,15
2,0,2,1
4,2,2,1
6,3,3,3


In [23]:
# South Africa Innings 2 powerplay runs_batter value counts
pp_1 = pd.DataFrame(df_1[df_1.over < 6].runs_batter.value_counts())
# South Africa Innings in middle overs
md_1 = pd.DataFrame(df_1[(df_1.over >= 6) & (df_1.over <= 15)].runs_batter.value_counts())
# # South Africa Innings in death overs

dt_1 = pd.DataFrame(df_1[df_1.over > 15].runs_batter.value_counts())

In [24]:
# merge pp_1, md_1, dt_1 dataframes
runs_summary = pp_1.merge(md_1, left_index=True, right_index=True, how='outer', suffixes=('_PP', '_MD'))
runs_summary = runs_summary.merge(dt_1, left_index=True, right_index=True, how='outer')
runs_summary = runs_summary.rename(columns={0: 'DT'})
runs_summary = runs_summary.fillna(0).astype(int)
runs_summary

,count_PP,count_MD,count
runs_batter,,,
0,17,21,7
1,12,28,10
2,0,8,4
4,5,2,2
6,2,1,1


In [193]:
6*2 + 10 * 4 + 2 * 5 + 1 * (26+9 + 9)

106